### Importing Libraries

In [127]:
import numpy as np
import pandas as pd

## Importing Dataset

In [128]:
dataset = pd.read_excel("HMF.xlsx")

## Modifying Columns

In [129]:
dataset[['Launch_Date']] = dataset[['Launch_Date']].astype("string")

In [130]:
def extract_year(arg):
    if(arg.item == 'None'):
        return
    else:
        # return arg.str.slice(0, 4)
        years = arg.str.slice(0, 4)
        return years

In [131]:
dataset = dataset.apply(lambda x: extract_year(x) if(x.name == 'Launch_Date') else x)

In [132]:
dataset[['Launch_Date']] = dataset[['Launch_Date']].astype("int")

In [133]:
def extract_age(arg):
    if(arg.item == 'None'):
        return
    else:
        age = 2022 - arg
        return age

In [134]:
dataset = dataset.apply(lambda x: extract_age(x) if(x.name == 'Launch_Date') else x)

## Preview

In [135]:
dataset.head()

,Category,Launch_Date,Crisil_Rating,Net_Assets,1_Yr_Return,3_Yr_Return,5_Yr_Return,Risk,Expense_Ratio,Sharpe_Ratio,Sortino_Ratio,Alpha,Beta,Std_Dev,R_Squared,Year_Trailing,Net_Return,Good
0,Arbitrage,16,Rank 1,1397.70,6.52,6.24,6.60,Moderately Low,0.83,-0.03,-0.04,0.60,0.33,0.45,0.00,13,7.40,0
1,Conservative,19,Rank 1,84.93,7.68,5.92,6.55,Moderate,2.30,-0.08,-0.12,-1.91,0.58,3.07,62.19,16,7.73,0
2,Conservative,18,Rank 1,1589.42,8.40,9.33,9.92,Moderately High,2.24,0.84,1.36,1.91,1.58,3.57,88.45,15,10.06,1
3,Arbitrage,12,Rank 2,8535.00,6.64,6.41,6.86,Moderately Low,1.04,0.26,0.45,0.70,1.30,0.54,85.00,8,17.83,1
4,Arbitrage,17,Rank 2,13360.00,6.56,6.32,6.75,Moderately Low,0.97,0.14,0.23,0.56,1.26,0.45,87.45,13,18.45,1


In [136]:
dataset.shape

(30, 18)

In [137]:
dataset.dtypes

Category          object
Launch_Date        int32
Crisil_Rating     object
Net_Assets       float64
1_Yr_Return      float64
3_Yr_Return      float64
5_Yr_Return      float64
Risk              object
Expense_Ratio    float64
Sharpe_Ratio     float64
Sortino_Ratio    float64
Alpha            float64
Beta             float64
Std_Dev          float64
R_Squared        float64
Year_Trailing      int64
Net_Return       float64
Good               int64
dtype: object

In [138]:
dataset.isnull().sum()

Category         0
Launch_Date      0
Crisil_Rating    0
Net_Assets       0
1_Yr_Return      0
3_Yr_Return      0
5_Yr_Return      0
Risk             0
Expense_Ratio    0
Sharpe_Ratio     0
Sortino_Ratio    0
Alpha            0
Beta             0
Std_Dev          0
R_Squared        0
Year_Trailing    0
Net_Return       0
Good             0
dtype: int64

## Encoding Categorical Data

In [139]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
ohe.fit(dataset[['Category', 'Crisil_Rating', 'Risk']])

OneHotEncoder()

In [140]:
encoded = pd.DataFrame.sparse.from_spmatrix(ohe.transform(dataset[['Category', 'Crisil_Rating', 'Risk']]))

In [141]:
dataset.drop(['Category', 'Crisil_Rating', 'Risk'], axis='columns', inplace=True)

In [142]:
dataset = pd.concat([encoded, dataset], axis=1)

## Splitting the dataset into the Training set and Test set

In [143]:
from sklearn.model_selection import train_test_split
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Feature Scaling

In [144]:
# Scaling URL_LENGTH & NUMBER_SPECIAL_CHARACTERS

from sklearn.preprocessing import StandardScaler
# Scaling X_train
X_train_features = X_train[:, -2:]

scaler = StandardScaler().fit(X_train_features)
X_train_features = scaler.transform(X_train_features)

X_train[:, -2:] = X_train_features

# Scaling X_test
X_test_features = X_test[:, -2:]

scaler = StandardScaler().fit(X_test_features)
X_test_features = scaler.transform(X_test_features)

X_test[:, -2:] = X_test_features

## Training KNN Model

In [145]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 4, metric = 'minkowski', p = 3)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=4, p=3)

### Confusion Matrix

In [146]:
y_pred = classifier.predict(X_test)

In [147]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [4 0]]


0.5555555555555556

## Training Logistic Regression Model

In [148]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

C:\Users\shaur\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

### Predicting Test Set Results

In [150]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]]


### Confusion Matrix

In [151]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [1 3]]


0.8888888888888888

## Training SVM Model

In [152]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X_train, y_train)

SVC(kernel='linear', random_state=0)

### Predicting Test Results

In [153]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]]


### Confusion Matrix

In [154]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [0 4]]


1.0

## Training Kernel SVM Model

In [155]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, y_train)

SVC(random_state=0)

### Predicting Test Results

In [156]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 1]
 [0 1]]


### Confusion Matrix

In [157]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [4 0]]


0.5555555555555556

## Training Naive Bayes Model

In [158]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

### Predicting Test Result

In [159]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]]


### Confusion Matrix

In [160]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [0 4]]


1.0

## Training Descision Tree Classification Model

In [161]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

### Predicting Test Results

In [162]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]]


### Confusion Matrix

In [163]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [0 4]]


1.0

## Training Random Forest Classification Model

In [164]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=0)

### Predicting Test Results

In [165]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 1]]


### Confusion Matrix

In [166]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[5 0]
 [2 2]]


0.7777777777777778